# Import Packages

In [1]:
import pandas as pd
import numpy as np

# Load Data

In [2]:
input_df = pd.read_csv("./data/20240117_churn_data.csv", low_memory=False).drop_duplicates()

# Prepare Data

## Filter Customers

In [4]:
# Only keep customers that joined after welcome discount was introduced
input_df = input_df[input_df["year_initiation_policy"] > 2019]

# We need to filter out all the customers that churned 
input_df['non_relevant_churn'] = input_df.groupby('policy_nr_hashed')[['d_churn_between_prolongations', 'd_churn_cancellation']].transform('sum').sum(axis=1)
input_df = input_df[input_df['non_relevant_churn'] == 0]

## Create Relevant Columns

In [5]:
# A variable that says how long it took for a customer to churn (0 means he didn't churn yet, 5 means he churned 5 years after joining)
input_df["years_to_churn"] = input_df["years_since_policy_started"] * input_df["d_churn_around_prolongation"]

In [6]:
display(input_df)

,policy_nr_hashed,years_since_policy_started,year_initiation_policy,year_initiation_policy_version,year_end_policy,d_churn,d_churn_cancellation,d_churn_between_prolongations,d_churn_around_prolongation,premium_main_coverages,...,premium_change_mutation_9,mutation_10,premium_change_mutation_10,mutation_11,premium_change_mutation_11,mutation_12,premium_change_mutation_12,data_collection_date,non_relevant_churn,years_to_churn
11089,0WKxEpm,2,2020,2022.0,2022.0,0,0,0,0,1796.928,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-15,0,0
11168,ByZrEAd,2,2020,2022.0,2022.0,0,0,0,0,3998.400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-15,0,0
11209,zPmpJyK,3,2020,2023.0,2023.0,0,0,0,0,6297.088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-15,0,0
11283,kbdLPlV,2,2020,2022.0,2022.0,0,0,0,0,3197.376,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-15,0,0
11328,AoZzenw,1,2020,2021.0,2021.0,0,0,0,0,6701.184,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-15,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437491,pjmeapy,1,2023,2024.0,2024.0,1,0,0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-15,0,1
437492,0WKkgr6,1,2023,2024.0,2024.0,1,0,0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-15,0,1
437493,6rYlkkr,1,2023,2024.0,2024.0,1,0,0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-15,0,1
437494,a6R9qP5,1,2022,2023.0,2023.0,1,0,0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-15,0,1


In [ ]:
}